# pympp

> A toolkit for multimodal process prediction with neural networks in python.

## Install

```sh
git clone https://github.com/joLahann/pympp
cd pympp
pip install .
```

## How to use

**Process Prediction**


In [1]:
from pympp.process import *
from pympp.prediction.predict import *

In [2]:
log = import_log('./data/logs/csv/mppn_ds/BPIC12.csv')
log.head()

,event_id,resource,timestamp,activity,REG_DATE,AMOUNT_REQ
trace_id,,,,,,
173688,0,112.0,2011-09-30 22:38:44.546000+00:00,###start###,2011-10-01 00:38:44.546000+02:00,20000
173688,1,112.0,2011-09-30 22:38:44.546000+00:00,A_SUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,2,112.0,2011-09-30 22:38:44.880000+00:00,A_PARTLYSUBMITTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,3,112.0,2011-09-30 22:39:37.906000+00:00,A_PREACCEPTED_COMPLETE,2011-10-01 00:38:44.546000+02:00,20000
173688,4,112.0,2011-09-30 22:39:38.875000+00:00,W_Completeren aanvraag_SCHEDULE,2011-10-01 00:38:44.546000+02:00,20000


In [3]:
col = 'activity'
o = PPObj(log,procs = Categorify(),cat_names=col,y_names=col,splits=split_traces(log))
o.show()

#traces: 12616 #events: 249149


,activity
trace_id,
214361,2
214361,12
214361,9


In [4]:
dls = o.get_dls()
m = RNNwEmbedding(o)
train_res = train_validate(dls,m,epoch=5,metrics=accuracy_targ0,loss=cross_entropy_targ0,print_output=False,show_plot=False)

In [5]:
train_res

0.8209230303764343

**Anomaly Detection**

In [6]:
from pympp.anomaly.detect import *
from fastai.basics import *
from sklearn.metrics import f1_score

In [7]:
log = import_log('./data/logs/csv/dapnn_ds/PDC2020_training/pdc_2020_0101100.csv.gz')
log.head()

,activity,event_id
trace_id,,
trace 1,###start###,0
trace 1,t31,1
trace 1,t41,2
trace 1,t26,3
trace 1,t36,4


In [8]:
o,dls,categorify = training_dl(log)

In [10]:
m=ControlFlowModel(o)
train_res =  train_validate(dls,m,epoch=25,metrics=accuracy_targ0,loss=cross_entropy_targ0,print_output=False,show_plot=False)
train_res 

(#2) [1.0479528903961182,0.5110132098197937]

In [11]:
test_log = import_log_pdc('../data/logs/csv/dapnn_ds/PDC2020_ground_truth/pdc_2020_0101100.csv.gz')
test_log.head()

,activity,case:pdc:costs,normal,event_id
trace_id,,,,
trace 1,###start###,0.0,True,0
trace 1,t21,0.0,True,1
trace 1,t32,0.0,True,2
trace 1,t26,0.0,True,3
trace 1,t41,0.0,True,4


In [12]:
o = process_test(test_log,categorify)
wds,idx=windows_fast(o.xs, o.event_ids)
res=(m(LongTensor(wds).cuda()))
anomaly_score = calc_anomaly_score(res,o,idx)
y_pred, y_true = get_anomalies(anomaly_score,o,idx)
f1_score(y_true, y_pred)

0.8571428571428571

Build with nbdev and fastai.